In [3]:
!wget https://data-flair.s3.ap-south-1.amazonaws.com/Data-Science-Data/face-mask-dataset.zip


In [4]:
import os
import zipfile

CWD = os.getcwd()
with zipfile.ZipFile(CWD+"/face-mask-dataset.zip","r") as zip_ref:
  zip_ref.extractall()

with zipfile.ZipFile(CWD+"/Dataset/train.zip","r") as zip_ref:
  zip_ref.extractall()

with zipfile.ZipFile(CWD+"/Dataset/test.zip","r") as zip_ref:
  zip_ref.extractall()

In [5]:
import glob
with_mask = glob.glob(CWD+'/train/with_mask/*')
without_mask = glob.glob(CWD+'/train/without_mask/*')
print(f"Train dataset: has {len(with_mask)} with mask images")
print(f"Train dataset: has {len(without_mask)} without mask images")

print(f"Test dataset: has {len(glob.glob(CWD+'/test/with_mask/*'))} with mask images")
print(f"Test dataset: has {len(glob.glob(CWD+'/train/without_mask/*'))} with mask images")

In [6]:
import matplotlib.pyplot as plt
import cv2

In [7]:
image = cv2.imread(with_mask[0])
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.title("With Mask")
plt.imshow(image)

In [8]:
image = cv2.imread(without_mask[0])
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.title("Without Mask")
plt.imshow(image)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense


In [10]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [11]:
model.summary()

In [23]:
TRAIN_DIR = "./train"
train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                        rescale=1.0/255,
                                                                        rotation_range=40,
                                                                        width_shift_range=0.2,
                                                                        height_shift_range=0.2,
                                                                        shear_range=0.2,
                                                                        zoom_range=0.2,
                                                                        horizontal_flip=True,
                                                                        fill_mode='nearest',
                                                            
                                                                     )

train_generator = train_datagenerator.flow_from_directory(TRAIN_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

In [24]:
train_generator.class_indices.items()

In [13]:
TEST_DIR = "./test"
test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                        rescale=1.0/255,
                                                                        
                                                                    )

test_generator = test_datagenerator.flow_from_directory(TEST_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

In [22]:
test_generator.class_indices.items()

In [15]:
model.fit(train_generator,
                              epochs=10,
                              validation_data=test_generator
          )

In [16]:
cv2_base_dir = os.path.dirname(os.path.abspath(cv2.__file__))
haar_model = os.path.join(cv2_base_dir, 'data/haarcascade_frontalface_default.xml')

In [ ]:
model.save('model_facemask.h5')